In [ ]:
import random
import numpy as np
import torch


In [ ]:
from main_ae import AgeModel
from main_ae import START_AGE, END_AGE, NUM_AGE_GROUPS, LAMBDA_1, LAMBDA_2


In [ ]:
NUM_AGE_GROUPS = 9
VALIDATION_RATE = 0.1
num_ages = END_AGE - START_AGE + 1


In [ ]:
random.seed(2019)
np.random.seed(2019)
torch.manual_seed(2019)


# AIHub dataset


In [ ]:
import json
import pandas as pd

In [ ]:
from fpt.data import join_face_df
from fpt.path import DTFR

DATA_CATEGORY = "aihub_family"
face_df = join_face_df(DTFR, DATA_CATEGORY)

In [ ]:
# Age
age_df = pd.read_csv("/home/jupyter/data/dataframe/df_aihub_ages.csv", index_col=0)
face_df = face_df.join(age_df, on="target")

# Age group
ages = face_df[face_df.category == "Age"]
age_group_df = ages.key.str.split('_').map(lambda x: x[-1][0]).to_frame(name="age_group")
face_df = face_df.join(age_group_df)

# Update age
face_df.loc[age_group_df.index, "age"] = None

face_df

In [ ]:
RANGE_TO_MEDIAN = {
    "a": (1 + 6) / 2,
    "b": (7 + 12) / 2,
    "c": (13 + 19) / 2,
    "d": (20 + 30) / 2,
    "e": (31 + 45) / 2,
    "f": (46 + 55) / 2,
    "g": (56 + 66) / 2,
    "h": (67 + 80) / 2,
    "above": 90,
}
AGE_GROUPS = ["a", "b", "c", "d", "e", "f", "g", "h", "above"]


def age_to_age_groups(age):
    if age <= 6:
        return "a"
    if age <= 12:
        return "b"
    if age <= 19:
        return "c"
    if age <= 30:
        return "d"
    if age <= 45:
        return "e"
    if age <= 55:
        return "f"
    if age <= 66:
        return "g"
    if age <= 80:
        return "h"
    return "above"

GROUP_TO_INDEX = {group: index for index, group in enumerate(AGE_GROUPS)}

In [ ]:
# age
is_age_null = face_df['age'].isnull()
range_to_age = face_df.loc[is_age_null, 'age_group'].map(lambda x: RANGE_TO_MEDIAN[x])
face_df.loc[is_age_null, 'age'] = range_to_age

In [ ]:
# age group
is_age_group_null = face_df['age_group'].isnull()
age_groups = face_df.loc[is_age_group_null, 'age'].map(lambda x: age_to_age_groups(int(x)))
face_df.loc[is_age_group_null, 'age_group'] = age_groups
face_df.age = face_df.age.astype(int)

In [ ]:
face_df.info()

# Model


In [ ]:
model = AgeModel(num_ages, NUM_AGE_GROUPS)  # age_pred, age_group_pred

# Dataloader


In [ ]:
from data import NiaDataset
import numpy as np
from PIL import Image
import os


In [ ]:
from torch.utils.data import Dataset, DataLoader
import torchvision


In [ ]:
train_meta_path = "nia_cropped/train_0.npy"
test_meta_path = "nia_cropped/test_0.npy"


In [ ]:
batch_size = 16


## NiaDataset


### Train dataloader


In [ ]:
transforms_train = torchvision.transforms.Compose(
    [
        torchvision.transforms.ToPILImage(),
        torchvision.transforms.RandomApply(
            [
                torchvision.transforms.RandomAffine(degrees=10, shear=16),
                torchvision.transforms.RandomHorizontalFlip(p=1.0),
            ],
            p=0.5,
        ),
        torchvision.transforms.Resize((256, 256)),
        torchvision.transforms.RandomCrop((224, 224)),
        torchvision.transforms.ToTensor(),
    ]
)


In [ ]:
train_gen = NiaDataset(train_meta_path, transforms_train)


In [ ]:
train_loader = DataLoader(
    dataset=train_gen,
    batch_size=batch_size,
    shuffle=True,
    pin_memory=True,
    num_workers=0,
)


In [ ]:
train_iter = iter(train_loader)


In [ ]:
sample = next(train_iter)
sample.keys()


In [ ]:
for key in sample.keys():
    print(f"{key}:\t{sample[key][0]}")

In [ ]:
face_df.key

In [ ]:
idx = 1
file_name, _ = os.path.splitext(sample['file'][idx])
file_name in face_df.key

### Validation dataloader


In [ ]:
transforms = torchvision.transforms.Compose(
    [
        torchvision.transforms.ToPILImage(),
        torchvision.transforms.Resize((224, 224)),
        torchvision.transforms.ToTensor(),
    ]
)
val_gen = NiaDataset(test_meta_path, transforms)
val_loader = DataLoader(
    val_gen, batch_size=1, shuffle=False, pin_memory=True, num_workers=0
)


# Optimizer


In [ ]:
from torch import optim
import torch
from torch.optim import lr_scheduler


In [ ]:
LAMBDA_1 = 0.2
LAMBDA_2 = 0.05
START_AGE = 0
END_AGE = 90
learning_rate = 1e-3
epoch = 2


In [ ]:
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = lr_scheduler.MultiStepLR(optimizer, milestones=[5, 8, 9], gamma=0.1)


# Loss function


In [ ]:
from mean_variance_loss import MeanVarianceLoss


In [ ]:
criterion1 = MeanVarianceLoss(LAMBDA_1, LAMBDA_2, START_AGE, END_AGE).cuda
criterion2 = torch.nn.CrossEntropyLoss().cuda()


# Train


In [ ]:
from main_ae import train_softmax, evaluate_softmax


In [ ]:
def train_softmax(train_loader, model, criterion2, optimizer, epoch, result_directory):
    model.cuda().train()
    running_loss = 0.0
    running_softmax_loss = 0.0
    interval = 1
    for i, sample in enumerate(train_loader):
        images = sample["image"].cuda()
        labels = sample["age_class"].cuda()
        _, output = model(images)
        loss = criterion2(output, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.data
        if (i + 1) % interval == 0:
            print("[%d, %5d] loss: %.3f" % (epoch, i, running_loss / interval))
            with open(os.path.join(result_directory, "log"), "a") as f:
                f.write("[%d, %5d] loss: %.3f\n" % (epoch, i, running_loss / interval))
            running_loss = 0.0


In [ ]:
for epoch in range(2):
    break
    train_softmax(train_loader, model, criterion2, optimizer, epoch, "result")
    loss_val, mae = evaluate_softmax(val_loader, model, criterion2)
    scheduler.step()


In [ ]:
# loss_val, mae


# Custom


In [ ]:
from torch import nn
from nia_age.main_ae import Embedding, AgeClassifier


In [ ]:
model = nn.Sequential(Embedding(), AgeClassifier(num_ages, NUM_AGE_GROUPS))
model = model.cuda()


In [ ]:
images = sample["image"]
images = images.cuda()


In [ ]:
pred_age, pred_age_group = model(images)
pred_age.shape, pred_age_group.shape


## Face Recognition


In [ ]:
from arcface_torch.losses import CombinedMarginLoss
from arcface_torch.configs.aihub_r50_onegpu import config as aihub_config
from arcface_torch.configs.base import config as cfg

cfg.update(aihub_config)
cfg.output = "work_dirs/aihub_r50_onegpu"

In [ ]:
margin_loss = CombinedMarginLoss(
    64,
    cfg.margin_list[0],
    cfg.margin_list[1],
    cfg.margin_list[2],
    cfg.interclass_filtering_threshold,
)

## AIHubDataset

In [ ]:
from facenet.datasets.AIHubDataset import AIHubDataset
from torchvision.datasets import ImageFolder
import torchvision

In [ ]:
root_dir = '/home/jupyter/data/face-image/train_aihub_family'
image_size = 112
aihub_mean = [0.5444, 0.4335, 0.3800]
aihub_std = [0.2672, 0.2295, 0.2156]
face_age_transform_train = transforms.Compose(
    [
        torchvision.transforms.Resize(size=(image_size, image_size)),
        torchvision.transforms.RandomHorizontalFlip(),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize(mean=aihub_mean, std=aihub_std),
    ]
)

In [ ]:
train_set = ImageFolder(root_dir, face_age_transform_train)
next(iter(train_set))

## Multi task Dataset


In [ ]:
import os
import torch
from torch.utils.data import Dataset
import torchvision
from torchvision.datasets import ImageFolder
from torchvision.transforms import transforms
from easydict import EasyDict as edict

In [ ]:
root_dir = '/home/jupyter/data/face-image/train_aihub_family'
image_size = 112
aihub_mean = [0.5444, 0.4335, 0.3800]
aihub_std = [0.2672, 0.2295, 0.2156]
face_age_transform_train = torchvision.transforms.Compose(
    [
        torchvision.transforms.Resize(size=(image_size, image_size)),
        torchvision.transforms.RandomHorizontalFlip(),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize(mean=aihub_mean, std=aihub_std),
    ]
)

In [ ]:
class FaceAgeDataset(Dataset):
    def __init__(self, root_dir, face_df, transform):
        self.face_dataset = ImageFolder(root=root_dir, transform=transform)
        self.face_df = face_df
        self.class_to_idx = self.face_dataset.class_to_idx

    def __len__(self):
        return len(self.face_dataset)

    def __getitem__(self, index):
        image, face_label = self.face_dataset[index]
        path, _ = self.face_dataset.samples[index]
        *_, key = os.path.splitext(path)[0].split("/")
        row = face_df.loc[key]
        sample = edict({
            "image": image,
            "age": row.age,
            "age_class": GROUP_TO_INDEX[row.age_group],
            "file": path,
            "data_type": row.category,
            "family_id": row.family_id,
            "personal_id": row.target,
            "face_label": face_label,
            "key": key,
        })
        return sample

In [ ]:
root_dir = "/home/jupyter/data/face-image/train_aihub_family"
face_age_dataset = FaceAgeDataset(root_dir, face_df, face_age_transform_train)
iterator = iter(face_age_dataset)

In [ ]:
sample = next(iterator)
[print(f"{k}: {v}") for k, v in sample.items()]
pass

# DataLoader

In [ ]:
from torch.utils.data import DataLoader
from torch.utils.data.dataloader import default_collate


def custom_collate(batch):
    data = {
        key: default_collate([d[key] for d in batch])
        for key in batch[0]
        if key != "age"
    }
    age = torch.tensor(
        [d["age"] for d in batch], dtype=torch.float32
    )  # 'age' 필드를 텐서로 변환합니다.
    data["age"] = age
    return data


batch_size = 16
face_age_loader = DataLoader(
    dataset=face_age_dataset,
    batch_size=batch_size,
    shuffle=True,
)
train_iter = iter(face_age_loader)
sample = next(train_iter)

In [ ]:
print(sample.family_id)
print(sample.age_class)
print(sample.personal_id)
print(sample.face_label)
sample.image.shape, sample.age

## Model

### nia_age

In [ ]:
from main_ae import AgeModel, Embedding
from main_ae import END_AGE, START_AGE, NUM_AGE_GROUPS
NUM_AGES = END_AGE - START_AGE + 1

In [ ]:
model = AgeModel(NUM_AGES, NUM_AGE_GROUPS)
model = model.cuda()

In [ ]:
images = sample.image.cuda()

In [ ]:
age_pred, age_group_pred = model(images)
age_pred.shape, age_group_pred.shape

In [ ]:
embedding = Embedding()
embedding = embedding.cuda()
embedding = embedding.eval()

In [ ]:
nia_embedding_result = embedding(images)
nia_embedding_result.shape

### ArcFace

In [ ]:
from arcface_torch.backbones import get_model
from arcface_torch.configs.aihub_r50_onegpu import config as cfg

In [ ]:
backbone = get_model(
    cfg.network,
    dropout=0.0,
    fp16=False,
    num_features=512,
).cuda()

In [ ]:
model_weights = torch.load(
    f"/home/jupyter/family-photo-tree/utils/model/arcface/{cfg.network}/backbone.pth"
)
backbone.load_state_dict(model_weights)

In [ ]:
backbone = backbone.eval()

In [ ]:
arcface_result = backbone(images)
arcface_result.shape

### Age Classifier with ArcFace

In [ ]:
from main_ae import AgeClassifier

In [ ]:
age_classifier = AgeClassifier(NUM_AGES, NUM_AGE_GROUPS).cuda()

In [ ]:
pred_age_with_arcface, pred_age_group_with_arcface = age_classifier(arcface_result)
pred_age_with_arcface.shape, pred_age_group_with_arcface.shape

## Loss Function


### CrossEntropyLoss

In [ ]:
import torch

In [ ]:
criterion2 = torch.nn.CrossEntropyLoss().cuda()
age_loss = criterion2(age_pred, sample.age.cuda())
age_group_loss = criterion2(age_group_pred, sample.age_class.cuda()) 

### Mean-Variance Loss

In [ ]:
from mean_variance_loss import MeanVarianceLoss

In [ ]:
criterion1 = MeanVarianceLoss(LAMBDA_1, LAMBDA_2, START_AGE, END_AGE).cuda()
mean_loss, variance_loss = criterion1(age_pred, sample.age)

In [ ]:
loss = mean_loss + variance_loss + age_loss + 10 * age_group_loss
loss

### CombinedMarginLoss

In [ ]:
from arcface_torch.losses import ArcFace, CombinedMarginLoss
from torch.nn.functional import normalize, linear
from torch.nn import CrossEntropyLoss
from torch.nn import NLLLoss  # Negative Log Likelihood

In [ ]:
margin_loss = CombinedMarginLoss(
    64,
    cfg.margin_list[0],
    cfg.margin_list[1],
    cfg.margin_list[2],
    cfg.interclass_filtering_threshold,
)
dist_cross_entropy = torch.nn.CrossEntropyLoss()
criterion = NLLLoss()

In [ ]:
# labels
labels = sample.face_label.cuda()
labels = labels.view(-1, 1)

In [ ]:
# embeddings
norm_embeddings = normalize(arcface_result)

In [ ]:
# weight
num_classes = len(face_age_dataset.class_to_idx)
weight = torch.nn.Parameter(torch.normal(0, 0.01, (num_classes, 512))).cuda()
print(weight.shape)
norm_weight_activated = normalize(weight)

In [ ]:
# logits
logits = linear(norm_embeddings, norm_weight_activated)
logits = logits.clamp(-1, 1)
logits = margin_loss(logits, labels)

In [ ]:
# loss
criterion3 = CrossEntropyLoss()
softmax = margin_loss(logits, labels)
loss3 = criterion3(softmax, labels.flatten())
loss3